In [ ]:
import pennylane as qml
import numpy as np

accessKeyId = ""
secretAccessKey = ""

In [ ]:
dev = qml.device(
        "kq.emulator",
        wires=2,
        shots=2048,
        accessKeyId=accessKeyId,
        secretAccessKey=secretAccessKey,
)

In [ ]:
dev = qml.device("default.qubit", wires=2, shots=2048)

In [ ]:
@qml.qnode(dev, interface="autograd")
def circuit1(param):
    qml.RY(param[0], wires=0)
    qml.RY(param[1], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RY(param[2], wires=0)
    qml.RY(param[3], wires=1)
    return qml.expval(qml.Identity(0)@qml.Identity(1))

In [ ]:
@qml.qnode(dev, interface="autograd")
def circuit2(param):
    qml.RY(param[0], wires=0)
    qml.RY(param[1], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RY(param[2], wires=0)
    qml.RY(param[3], wires=1)
    return qml.expval(qml.Identity(0)@qml.PauliX(1))

In [ ]:
@qml.qnode(dev, interface="autograd")
def circuit3(param):
    qml.RY(param[0], wires=0)
    qml.RY(param[1], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RY(param[2], wires=0)
    qml.RY(param[3], wires=1)
    return qml.expval(qml.PauliX(0)@qml.PauliX(1))

In [ ]:
@qml.qnode(dev, interface="autograd")
def circuit4(param):
    qml.RY(param[0], wires=0)
    qml.RY(param[1], wires=1)
    qml.CNOT(wires=[0,1])
    qml.RY(param[2], wires=0)
    qml.RY(param[3], wires=1)
    return qml.expval(qml.PauliY(0)@qml.PauliY(1))

In [ ]:
# calculate the cost function
def cost_fn(param):
    return 2*circuit1(param) - circuit2(param) - 0.5*circuit3(param) - 0.5*circuit4(param)

In [ ]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)
theta = qml.numpy.array([0.5*np.pi, 0.5*np.pi, 0.5*np.pi, 0.5*np.pi], requires_grad=True)

# store the values of the cost function
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 100
conv_tol = 1e-04

In [ ]:
for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)
    energy.append(cost_fn(theta))
    angle.append(theta)
    conv = np.abs(energy[-1] - prev_energy)
    
    if conv <= conv_tol:
        break

In [ ]:
import matplotlib.pyplot as plt

# Exact energy
H1 = np.kron(np.array([[1, 0], [0, 1]]), np.array([[1, 0], [0, 1]]))
H2 = np.kron(np.array([[1, 0], [0, 1]]), np.array([[0, 1], [1, 0]])) 
H3 = np.kron(np.array([[0, 1], [1, 0]]), np.array([[0, 1], [1, 0]]))
H4 = -np.kron(np.array([[0, -1], [1, 0]]), np.array([[0, -1], [1, 0]]))
H = 2*H1 - H2 - 0.5*H3 - 0.5*H4
exact_energy = np.min(np.linalg.eig(H)[0])

# plot
plt.plot(range(len(energy)), energy, "bo", markersize=3, ls="dashed", label='VQE')
plt.plot(range(len(energy)), np.full(len(energy), exact_energy), color="red", label='Exact')
plt.xlabel("Optimization step", fontsize=13)
plt.ylabel("Energy", fontsize=13)
plt.legend()
plt.show()